### Part1-A: Import Panda and Beautiful Soup

In [1]:
##### import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Part1-B: Using BeautifulSoup for webpage scraping and parsing Toronto Neighborhood table

In [2]:
website_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

import urllib.request
html_file = urllib.request.urlopen(website_url)

soup = BeautifulSoup(html_file)

tables = soup.findAll('table')
first_table = soup.find("table", class_ = "wikitable sortable")

Col1 = []
Col2 = []
Col3 = []

#Load the table from website
for row in first_table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells)==3: #Only extract table body not heading
        Col1.append(cells[0].find(text=True))
        Col2.append(cells[1].find(text=True))
        Col3.append(cells[2].find(text=True))

### Part1-C: Dataframe and cleaning

In [3]:
#change the table to a dataframe
df=pd.DataFrame(Col1,columns=["PostalCode"])
df["Borough"]=Col2
df["Neighborhood"]=Col3

In [4]:
def drop_n(test):
    test=test.split("\n")[0]
    return test
#drop the \n character
df["PostalCode"]=df["PostalCode"].apply(drop_n)
df["Borough"]=df["Borough"].apply(drop_n)
df["Neighborhood"]=df["Neighborhood"].apply(drop_n)

In [5]:
# drop a not assigned Borough
df=df[df["Borough"]!="Not assigned"]
df=df.reset_index()
df=df[["PostalCode","Borough","Neighborhood"]]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

### Part2-A: Adding latitude and the longitude coordinates of each neighborhood

In [7]:
df_Coordinate = pd.read_csv("http://cocl.us/Geospatial_data")
df_Coordinate = df_Coordinate.rename({'Postal Code': 'PostalCode'}, axis=1)  # new method

In [8]:
df_combined_toronto = pd.merge(df,
                 df_Coordinate[["PostalCode","Latitude","Longitude"]],
                 on='PostalCode')
df_combined_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part2-B: Using geocoder, very unreliable

In [9]:
#%pip install geocoder
import geocoder # import geocoder

for n in range(df.shape[0]):
    Postal_Code = df['PostalCode'][n]
    print(Postal_Code)
    
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
          g = geocoder.google('{}, Toronto, Ontario'.format(Postal_Code))
          lat_lng_coords = g.latlng

    df_Coordinate['Latitude'][n]  = lat_lng_coords[0]
    df_Coordinate['Longitude'][n] = lat_lng_coords[1]

df_Coordinate.head()

M3A


KeyboardInterrupt: 